# Aufbereitung der Stellenanzeige (rohe Textdaten)

In [131]:
import pandas as pd
import re
import nltk
import numpy as np
# from nltk.corpus import stopwords
# from nltk.tokenize import sent_tokenize, word_tokenize
import ast # benötigt um sections von str in list umzuwandeln!
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.display import display, HTML

In [132]:
display(HTML("<style>.container { width:75% !important; }</style>")) # Breite des Notebooks 
pd.set_option('display.max_rows', None)

stop_words = set(stopwords.words('german'))
print(stopwords)

## Daten laden und betrachten

In [133]:
df = pd.read_csv("df_1-3872_2019-07-20.csv", sep=";")

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50458 entries, 0 to 50457
Data columns (total 27 columns):
title                     50458 non-null object
company                   50458 non-null object
company_link              50458 non-null object
location                  50458 non-null object
contract_type             50458 non-null object
wrk_time                  50458 non-null object
date                      50458 non-null object
job_url                   50458 non-null object
sections                  50458 non-null object
urls                      50458 non-null object
searches                  50458 non-null object
job_count                 50458 non-null int64
search                    50458 non-null object
function                  50458 non-null int64
region                    50458 non-null int64
company_name              49887 non-null object
company_location          49649 non-null object
company_sector            22256 non-null object
company_rating            2043

# Unique ID generieren (laufende Nummer)

In [135]:
df.insert(0, 'unique_id', range(1, 1 + len(df)))

In [136]:
df.head()

,unique_id,title,company,company_link,location,contract_type,wrk_time,date,job_url,sections,...,company_sector,company_rating,company_rating_count,company_job_count,company_employee_count,company_website,company_metadatablock,function_label_level2,function_label_level1,function_id_level_1
0,1,Automatisierer (w/m/d),Berliner Wasserbetriebe,https://www.stepstone.de/cmp/de/Berliner-Wasserbetriebe-53041/jobs.html,Berlin,Feste Anstellung,Vollzeit,2019-07-18,https://www.stepstone.de/stellenangebote--Automatisierer-w-m-d-Berlin-Berliner-Wasserbetriebe--5922324-inline.html?suid=053e002f-6058-42bf-b733-45f2e0b16ff6&rltr=1_1_100_dynrl_m_0_0_0,['Über die Berliner Wasserbetriebe Die Berliner Wasserbetriebe sind mit 4.382 Mitarbeiterinnen und Mitarbeitern das größte Unternehmen der Wasser- und Abwasserbranche in Deutschland. Als Unternehm...,...,Energie- und Wasserversorgung & Entsorgung,4.6,17.0,23.0,0-50,http://netzdermöglichkeiten.de,"<div data-block=""app-inShort"" data-brand='{""primary"":""#8ab925"",""secondary"":""#0072c6""}' data-companyid=""53041"" data-configs='{""environment"":""live"",""debug"":false,""country"":""de"",""site"":""stepstone"",""d...",Automatisierungstechnik,Ingenieure und technische Berufe,10000000
1,2,Consultant (m/w/d) Robotic Process Automation (RPA) / Digitalisierung / Automatisierung,STAUFEN.AG,https://www.stepstone.de/cmp/de/STAUFEN-AG-30219/jobs.html,Großräume Stuttgart Köln München Nürnberg Ulm Frankfurt Mannheim Dortmund Hannover Hamburg Berlin,Feste Anstellung,Vollzeit,2019-07-17,https://www.stepstone.de/stellenangebote--Consultant-m-w-d-Robotic-Process-Automation-RPA-Digitalisierung-Automatisierung-Grossraeume-Stuttgart-Koeln-Muenchen-Nuernberg-Ulm-Frankfurt-Mannheim-Dort...,['In jedem Unternehmen steckt ein noch besseres. Mit dieser Überzeugung berät und qualifiziert die Staufen AG seit über 20 Jahren Unternehmen und Mitarbeiter weltweit. Was es dazu braucht? Versier...,...,"Unternehmensberatg., Wirtschaftsprüfg., Recht",NaN,NaN,6.0,320,http://www.staufen.ag,"<div data-block=""app-inShort"" data-brand='{""primary"":"""",""secondary"":""#003c72""}' data-companyid=""30219"" data-configs='{""environment"":""live"",""debug"":false,""country"":""de"",""site"":""stepstone"",""dataLang...",Automatisierungstechnik,Ingenieure und technische Berufe,10000000
2,3,Ingenieur zur Automatisierung des Bahnbetriebs (w/m/d),Deutsche Bahn AG,https://www.stepstone.de/cmp/de/Deutsche-Bahn-AG-60544/jobs.html,Berlin,Feste Anstellung,Vollzeit,2019-07-17,https://www.stepstone.de/stellenangebote--Ingenieur-zur-Automatisierung-des-Bahnbetriebs-w-m-d-Berlin-Deutsche-Bahn-AG--5920598-inline.html?suid=053e002f-6058-42bf-b733-45f2e0b16ff6&rltr=3_3_100_d...,['Einleitung Du willst an einem historisch einmaligen Vorhaben mitwirken und Deutschlands Bahnbetrieb in die digitale Zukunft begleiten? Hier kannst Du es - als Ingenieur zur Automatisierung des B...,...,Transport & Logistik,3.6,658.0,1040.0,200.000,https://www.deutschebahn.com,"<div data-block=""app-inShort"" data-brand=""{}"" data-companyid=""60544"" data-configs='{""environment"":""live"",""debug"":false,""country"":""de"",""site"":""stepstone"",""dataLanguagesMapping"":{""en"":""de"",""de"":""de""...",Automatisierungstechnik,Ingenieure und technische Berufe,10000000
3,4,Softwareentwickler (m/w/d) Automatisierungstechnik,HEITEC AG,https://www.stepstone.de/cmp/de/HEITEC-AG-39185/jobs.html,Berlin,Feste Anstellung,Vollzeit,2019-07-15,https://www.stepstone.de/stellenangebote--Softwareentwickler-m-w-d-Automatisierungstechnik-Berlin-HEITEC-AG--5915281-inline.html?suid=053e002f-6058-42bf-b733-45f2e0b16ff6&rltr=4_4_100_dynrl_m_0_0_0,"['HEITEC steht seit über 40 Jahren für Industriekompetenz in Automatisierung, Digitalisierung und Elektronik und bietet Lösungen, Produkte und Dienstleistungen. Mehr als 1000 Mitarbeiter an zahlre...",...,"Elektrotechnik, Feinmechanik & Optik Maschinen- und Anlagenbau Sonstige Dienstleistungen",3.9,6.0,23.0,1.000+,http://www.heitec.de,"<div data-block=""a

# Job ID extrahieren aus Job URL

In [137]:
df["job_id"] = [re.search(r'(\d{7})', url).group(1) for url in df["job_url"]]

In [138]:
df["idlength"] = [len(x) for x in df["job_id"]]
print(df["idlength"].value_counts())
df.drop("idlength", axis=1, inplace=True)

7    50458
Name: idlength, dtype: int64


In [139]:
# df["job_id"].value_counts() 

In [140]:
df.job_id.isnull().sum() # keine missings

0

In [141]:
# Unikate?:
len(df['job_id'].unique())
# einige job_ids kömmen häufiger vor. 
# (entsprechen ca 15000 Duplikaten ) 
# verbleiben vorerst im Sample, weil sie womöglich über unterschiedliche Berufsfelder gefunden wurden,
# bei Regression clustern!?

35229

# Sections in 6 Spalten zerteilen: 

In [142]:
type(df['sections'][1])

str

In [143]:
# Messen wie viele Sections es maximal gibt:
df['sectionlength'] = df.apply(lambda row: len(ast.literal_eval(row.sections)), axis = 1) 
df[["sectionlength", "sections"]].head(2)

,sectionlength,sections
0,5,['Über die Berliner Wasserbetriebe Die Berliner Wasserbetriebe sind mit 4.382 Mitarbeiterinnen und Mitarbeitern das größte Unternehmen der Wasser- und Abwasserbranche in Deutschland. Als Unternehm...
1,5,['In jedem Unternehmen steckt ein noch besseres. Mit dieser Überzeugung berät und qualifiziert die Staufen AG seit über 20 Jahren Unternehmen und Mitarbeiter weltweit. Was es dazu braucht? Versier...


In [144]:
df.sectionlength.value_counts()
# 7 Jobs haben keine Section mit Stellenbeschreibung -> müssen entfernt weren

5    39936
4     6359
1     3112
3      575
2      416
6       53
0        7
Name: sectionlength, dtype: int64

In [145]:
print(df.loc[df['sectionlength'] == 0].job_url)

17377    https://www.stepstone.de/stellenangebote--Systemadminstrator-m-w-d-Email-Kommunikation-Outlook-Nuernberg-DATEV-eG--5913831-inline.html?suid=1dfac600-b2ea-4934-87c4-6ae9f791d13d&rltr=131_31_100_dyn...
42372    https://www.stepstone.de/stellenangebote--Elektroniker-m-w-divers-in-dem-Bereich-Instandhaltung-Hagen-thyssenkrupp-Federn-und-Stabilisatoren-GmbH--5932045-inline.html?suid=e6ef12da-b901-444f-954f-...
42460    https://www.stepstone.de/stellenangebote--Elektroniker-m-w-divers-in-dem-Bereich-Instandhaltung-Hagen-thyssenkrupp-Federn-und-Stabilisatoren-GmbH--5932045-inline.html?suid=33597cb7-63db-4d3b-a197-...
42529    https://www.stepstone.de/stellenangebote--Instandhaltungsplaner-m-w-divers-befristet-fuer-1-Jahr-Hagen-thyssenkrupp-Federn-und-Stabilisatoren-GmbH--5932022-inline.html?suid=8d0fc8e6-769e-4434-9c19...
42641    https://www.stepstone.de/stellenangebote--Instandhaltungsplaner-m-w-divers-befristet-fuer-1-Jahr-Hagen-thyssenkrupp-Federn-und-Stabilisatoren-GmbH--5932022

In [146]:
# Die 7 Stellen ohne Section mit Stellenbeschreibung werden gelöscht
df.drop(df[df['sectionlength'] == 0].index, inplace=True)

In [147]:
df.shape

(50451, 30)

In [148]:
# sections Spalte (ist aktuell noch einzelner str) in Liste von strings umwandeln --> section_list  
df['section_list'] = df.apply(lambda row: ast.literal_eval(row.sections), axis = 1) 

In [149]:
df["section_list"][1]

['In jedem Unternehmen steckt ein noch besseres. Mit dieser Überzeugung berät und qualifiziert die Staufen AG seit über 20 Jahren Unternehmen und Mitarbeiter weltweit. Was es dazu braucht? Versierte Berater und Strategen, die im Lean Management zuhause sind. 320 Mitarbeiter an 13 internationalen Standorten sind bereits rund um den Globus unterwegs und helfen dabei, die richtigen Veränderungen bei unseren Kunden in Gang zu bringen. Und jetzt sind Sie dran: Nutzen Sie Ihre Chance, Großes zu leisten, um im großen Stil voranzukommen – bei STAUFEN. Verstärken Sie unser Team bundesweit und arbeiten Sie projektbezogen vor Ort bei unseren Kunden (z. B. rund um die Großräume Stuttgart, Köln, München, Nürnberg, Ulm, Frankfurt, Mannheim, Dortmund, Hannover, Hamburg, Berlin) als Consultant (m/w/d) Robotic Process Automation (RPA) / Digitalisierung / Automatisierung',
 'Ihre Herausforderung. Vielseitig. Mit Ihrem Know-how fokussieren Sie sich auf die Automatisierung und Digitalisierung der Prozesse

In [150]:
# section_list in einzelne Spalten überführen:
df[['section1','section2','section3','section4','section5','section6']] = pd.DataFrame(df.section_list.values.tolist(), index= df.index)

In [151]:
# sections umbenennen in sections_str (in Abgrenzung zu sections_list)
df.rename(columns={'sections':'sections_str'}, inplace=True)

# Berufsfelder / Funktionen
* beide Funktionslevel kombinieren z.B. Führungskraft - sonstige Führungskräfte
* Export to Excel
* Funktionen manuell mit KldB abgleichen mit BERUFENET der Arbeitsagentur

In [152]:
df['function_both_levels'] = df['function_label_level2'].astype(str) + " - " + df['function_label_level1'].astype(str)

In [153]:
pd.options.display.max_colwidth = 200
# pd.options.display.max_rows = 300
df['function_both_levels'].value_counts()

Teamleitung, Gruppenleitung - Führungskräfte                                                                1909
Gastronomie, Hotellerie - Handwerk, Dienstleistung und Fertigung                                            1652
Softwareentwicklung - IT                                                                                    1626
Außendienst - Vertrieb und Verkauf                                                                          1583
Consulting, Engineering - IT                                                                                1517
Projektmanagement - Ingenieure und technische Berufe                                                        1449
Elektrotechnik, Elektronik - Ingenieure und technische Berufe                                               1092
Projektmanagement - IT                                                                                      1064
Systemadministration, Netzwerkadministration - IT                                               

In [154]:
# Berufsfelder zur Klassifikation exportieren (nur einmal ausführen):
# pd.DataFrame(df['function_both_levels'].value_counts()).to_excel("function_both_levels.xlsx") 

In [155]:
df[['function_both_levels']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50451 entries, 0 to 50457
Data columns (total 1 columns):
function_both_levels    50451 non-null object
dtypes: object(1)
memory usage: 788.3+ KB


## Klassifikation der Berufe mergen

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50451 entries, 0 to 50457
Data columns (total 38 columns):
unique_id                 50451 non-null int64
title                     50451 non-null object
company                   50451 non-null object
company_link              50451 non-null object
location                  50451 non-null object
contract_type             50451 non-null object
wrk_time                  50451 non-null object
date                      50451 non-null object
job_url                   50451 non-null object
sections_str              50451 non-null object
urls                      50451 non-null object
searches                  50451 non-null object
job_count                 50451 non-null int64
search                    50451 non-null object
function                  50451 non-null int64
region                    50451 non-null int64
company_name              49880 non-null object
company_location          49642 non-null object
company_sector            22255

### Anhand von Stepstone Berufsfeldern/Funktionen (manuell in Excel)

In [157]:
df_kldb10 = pd.read_excel("function_both_levels_KldB10.xlsx")
df_kldb10.head()

,function_both_levels,function_both_levels_count,Führungskraft,KldB10-2,KldB10-2_label,KldB10-3,KldB10-3_label,KldB10-4,KldB10-4_label
0,"Teamleitung, Gruppenleitung - Führungskräfte",1909,1,71,Berufe in Unternehmensführung und -organisation,713.0,Unternehmensorganisation und -strategie,7139.0,Aufsichts- und Führungskräfte – Unternehmensorganisation und -strategie
1,"Gastronomie, Hotellerie - Handwerk, Dienstleistung und Fertigung",1652,0,63,"Tourismus-, Hotel- und Gaststättenberufe",633.0,Gastronomie,NaN,NaN
2,Softwareentwicklung - IT,1626,0,43,"Informatik-, Informations- und Kommunikationstechnologieberufe",434.0,Softwarentwicklung und Programmierung,4341.0,Berufe in der Softwareentwicklung
3,Außendienst - Vertrieb und Verkauf,1583,0,61,"Einkaufs-, Vertriebs- und Handelsberufe",611.0,Einkauf und Vertrieb,6112.0,Berufe im Vertrieb
4,"Consulting, Engineering - IT",1517,0,43,"Informatik-, Informations- und Kommunikationstechnologieberufe",432.0,"IT-Systemanalyse, IT-Anwendungsberatung und IT-Vertrieb",4322.0,Berufe in der IT-Anwendungsberatung


In [158]:
df_kldb10["KldB10-2"].dtype

dtype('int64')

In [159]:
df_kldb10["KldB10-2"] = df_kldb10["KldB10-2"].astype("int")

In [160]:
df = pd.merge(df, df_kldb10[["function_both_levels", "function_both_levels_count", "Führungskraft", "KldB10-2", "KldB10-3", "KldB10-4"]], on='function_both_levels', how='left')

In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50451 entries, 0 to 50450
Data columns (total 43 columns):
unique_id                     50451 non-null int64
title                         50451 non-null object
company                       50451 non-null object
company_link                  50451 non-null object
location                      50451 non-null object
contract_type                 50451 non-null object
wrk_time                      50451 non-null object
date                          50451 non-null object
job_url                       50451 non-null object
sections_str                  50451 non-null object
urls                          50451 non-null object
searches                      50451 non-null object
job_count                     50451 non-null int64
search                        50451 non-null object
function                      50451 non-null int64
region                        50451 non-null int64
company_name                  49880 non-null object
company_loc

In [162]:
df["KldB10-2"].value_counts()

43      10140
71       8914
72       5250
61       5167
27       4886
92       2171
26       1923
25       1691
63       1652
51       1411
81       1142
62        986
73        929
9999      723
31        688
41        569
83        419
34        360
82        293
23        249
84        212
53        124
24        123
29        109
32         99
91         94
22         45
93         28
42         19
33         18
28          9
94          8
Name: KldB10-2, dtype: int64

In [163]:
df["KldB10-2"] = df["KldB10-2"].astype(int)

In [164]:
df["KldB10-2"] = df["KldB10-2"].replace(9999, np.nan)

In [165]:
df["KldB10-2"].isnull().sum()

723

In [166]:
# Datentyp ändern in integer:
df["KldB10-3"] = pd.DataFrame(df["KldB10-3"], dtype=pd.Int64Dtype())

In [167]:
df["KldB10-3"].isnull().sum() #.value_counts()

1429

### Nachklassifizierung des "Rests"  (manuell in Excel)

In [168]:
df_kldb10_rest = pd.read_csv("df_kldb10_rest_ersatz.csv", sep=";")

In [169]:
df_kldb10_rest.rename(columns={'KldB10-2':'KldB10-2_rest', 
                              'KldB10-3': 'KldB10-3_rest'
                              }, inplace=True)

In [170]:
df_kldb10_rest.head(2)

,unique_id,KldB10-2_rest,KldB10-3_rest
0,4627,26.0,263.0
1,4628,26.0,263.0


In [171]:
df_kldb10_rest.isnull().sum()


unique_id          0
KldB10-2_rest    202
KldB10-3_rest    296
dtype: int64

In [172]:
df = pd.merge(df, df_kldb10_rest, on='unique_id', how='left')

In [173]:
df.shape

(50451, 45)

In [174]:
df[["KldB10-3_rest", "KldB10-3"]].info() #.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50451 entries, 0 to 50450
Data columns (total 2 columns):
KldB10-3_rest    1133 non-null float64
KldB10-3         49022 non-null object
dtypes: float64(1), object(1)
memory usage: 1.2+ MB


In [175]:
# Spalten zusammenführen
df['KldB10-3'] = np.where(df['KldB10-3_rest'].notna(), df['KldB10-3_rest'], df['KldB10-3'] )

In [176]:
df["KldB10-3"].value_counts()

611.0    4821
713.0    4342
433.0    4262
722.0    3021
271.0    2822
432.0    2706
714.0    2654
921.0    1874
715.0    1847
434.0    1791
273.0    1751
721.0    1749
633.0    1659
251.0    1425
431.0    1381
263.0    1210
516.0    1079
621.0     986
731.0     700
311.0     688
261.0     622
723.0     485
813.0     445
831.0     419
814.0     377
613.0     359
272.0     324
513.0     304
252.0     290
232.0     249
413.0     246
341.0     235
821.0     189
922.0     174
811.0     163
732.0     153
262.0     149
531.0     133
924.0     123
244.0     123
632.0     118
292.0     117
844.0     108
321.0      99
825.0      98
818.0      94
913.0      94
711.0      85
411.0      84
242.0      84
521.0      84
631.0      79
414.0      76
842.0      71
412.0      66
817.0      66
541.0      48
342.0      44
822.0      33
843.0      32
931.0      28
515.0      27
223.0      26
812.0      21
221.0      19
421.0      19
332.0      18
522.0      14
282.0       9
234.0       9
293.0       8
121.0 

In [177]:
df["KldB10-3"].isnull().sum()

296

In [178]:
df['KldB10-2'] = np.where(df['KldB10-2_rest'].notna(), df['KldB10-2_rest'], df['KldB10-2'] )

In [179]:
df['KldB10-2'].value_counts()

43.0    10140
71.0     8918
72.0     5255
61.0     5175
27.0     4907
92.0     2171
26.0     1932
63.0     1780
25.0     1709
51.0     1411
81.0     1146
62.0      989
73.0      929
31.0      688
41.0      566
83.0      426
34.0      368
82.0      325
23.0      259
84.0      217
24.0      205
53.0      131
29.0      125
32.0      101
52.0       98
91.0       94
54.0       48
22.0       45
93.0       28
42.0       19
33.0       18
28.0        9
94.0        8
12.0        7
11.0        2
Name: KldB10-2, dtype: int64

In [180]:
df['KldB10-2'].isnull().sum()

202

In [181]:
df.drop("KldB10-2_rest", axis=1, inplace=True)
df.drop("KldB10-3_rest", axis=1, inplace=True)

### KldB10-2 Labels

In [182]:
df_kldb10_2_label = pd.read_excel("kldb10-2_label.xlsx")

In [183]:
df_kldb10_2_label.head(2)

,KldB10-2,kldb10-2_label
0,11,"Land-, Tier-, Forstwirtschaftsberufe"
1,12,"Gartenbauberufe, Floristik"


In [184]:
df = pd.merge(df, df_kldb10_2_label, on='KldB10-2', how='left')
df.shape

(50451, 44)

### KldB10-3 Labels

In [185]:
df_kldb10_3_label = pd.read_excel("kldb10-3_label.xlsx")

In [186]:
df = pd.merge(df, df_kldb10_3_label, on='KldB10-3', how='left')

In [187]:
df.shape

(50451, 45)

# Frauenanteile aus Mikrozensus

## Frauenanteile in KldB10_2 aus Mikrozensus 2014 mergen

In [188]:
df_anteilMZ14_2 = pd.read_excel("Frauenanteil_MZ14_KldB10_2_collapsed.xlsx")

In [189]:
df_anteilMZ14_2.head(2)

,kldb10_2,frauenanteil_mz14_kldb10_2
0,1,0.088106
1,11,0.299710


In [190]:
df_anteilMZ14_2.rename(columns={'kldb10_2':'KldB10-2'}, inplace=True)

In [191]:
df = pd.merge(df, df_anteilMZ14_2, on='KldB10-2', how='left')

df.drop(["KldB10-2_label"],inplace=True, axis=1)

In [193]:
df.shape

(50451, 46)

## Frauenanteile kldb10_3 (MZ 2014) mergen

In [194]:
df_anteilMZ14 = pd.read_csv("Frauenanteil_MZ_KldB10-3_collapsed.csv", sep=";")
df_anteilMZ14.rename(columns={"kldb10_3": "KldB10-3"}, inplace=True)
df_anteilMZ14.head(2)

,KldB10-3,Frauenanteil_kldb10_3_MZ14,Anzahl_Frauen_MZ14
0,11.0,0.037736,53
1,12.0,0.100000,40


In [195]:
df.shape

(50451, 46)

In [196]:
df = pd.merge(df, df_anteilMZ14[['KldB10-3', 'Frauenanteil_kldb10_3_MZ14']], on='KldB10-3', how='left')

In [197]:
df["Frauenanteil_kldb10_3_MZ14"] = [round(x*100,2) for x in df["Frauenanteil_kldb10_3_MZ14"]]

In [198]:
df.shape

(50451, 47)

In [199]:
df.rename(columns={"Frauenanteil_kldb10_3_MZ14": "Frauenanteil_MZ14_kldb10_3",
                    "Frauenanteil_SOEP_kldb10-3": "Frauenanteil_SOEP_kldb10_3",
                  }, inplace=True)

In [201]:
df[["frauenanteil_mz14_kldb10_2", "Frauenanteil_MZ14_kldb10_3"]].describe()

,frauenanteil_mz14_kldb10_2,Frauenanteil_MZ14_kldb10_3
count,50249.000000,50155.000000
mean,0.413202,39.501637
std,0.238064,24.927755
min,0.026663,0.830000
25%,0.145675,13.250000
50%,0.367397,30.940000
75%,0.660914,65.840000
max,0.857164,98.150000


# Bundesland

In [202]:
df["Bundesland"] = df["region"].map(
    {419239: "Berlin", 
     423640: "Hamburg", 
     423047: "Bremen",
     200000088: "Schleswig-Holstein",
     200000092: "Sachsen",
     200000093: "Sachsen-Anhalt",
     200000094: "Thüringen",
     200000095: "Hessen",
     200000091: "Mecklenburg-Vorpommern",
     200000090: "Brandenburg",
     200000096: "Baden-Württemberg",
     200000081: "Bayern",
     200000084: "Nordrhein-Westfalen",
     200000082: "Saarland",
     200000083: "Rheinland-Pfalz",
     200000085: "Niedersachsen"})

In [203]:
df["Bundesland"].value_counts() #(normalize = True)

Nordrhein-Westfalen       9695
Baden-Württemberg         9175
Bayern                    8443
Hessen                    4997
Berlin                    4259
Hamburg                   4064
Niedersachsen             2960
Sachsen                   1394
Rheinland-Pfalz           1371
Schleswig-Holstein        1148
Bremen                     670
Brandenburg                566
Thüringen                  534
Mecklenburg-Vorpommern     482
Sachsen-Anhalt             474
Saarland                   219
Name: Bundesland, dtype: int64

In [204]:
# Anzahl der Level 2 - Funktionen, die "Mechatronik" beinhalten
df.title.loc[df['function_label_level2'] == "Mechatronik"].describe()

count                             390
unique                            311
top       KFZ - Mechatroniker (m/w/d)
freq                               19
Name: title, dtype: object

In [205]:
# Kreuztabelle der Job-Titel und Keywords, ggfs besser bedingt auf einzelnen Beruf
pd.crosstab(df["function_label_level2"], df["function_label_level1"])

function_label_level1,Administration und Sekretariat,"Banken, Finanzdienstleister und Versicherungen",Bildung und Soziales,"Design, Gestaltung und Architektur","Einkauf, Materialwirtschaft und Logistik",Finanzen,Führungskräfte,"Handwerk, Dienstleistung und Fertigung",IT,Ingenieure und technische Berufe,Marketing und Kommunikation,Naturwissenschaften und Forschung,Personal,"Pflege, Therapie und Assistenz",Recht,Vertrieb und Verkauf,Ärzte,Öffentlicher Dienst
function_label_level2,,,,,,,,,,,,,,,,,,
"(Bundes-)Polizei, Justizvollzug",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
"Abteilungsleitung, Bereichsleitung",0,0,0,0,0,0,870,0,0,0,0,0,0,0,0,0,0,0
Administration,0,0,0,0,0,0,0,0,0,0,0,0,244,0,0,0,0,0
Allgemeinmedizin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,0
"Altenpflege, Betreuungsberufe",0,0,0,0,0,0,0,0,0,0,0,0,0,76,0,0,0,0
Analyst,0,55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Angestellte, Beamte auf Bundesebene",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
"Angestellte, Beamte auf Landes-, kommunaler Ebene",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,73
"Anlageberatung, Vermögensberatung",0,66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Contract Type / Vertragsart
* Zwei neue Variablen erstellen: Befristung & Vertragsart

In [206]:
df["contract_type"].value_counts() #(normalize = True)
# hier werden zusammengefasst bzw. gesplittet in zwei Merkmale: Vertragsart und Befristung

Feste Anstellung                                                              42847
Praktikum                                                                      1699
Befristeter Vertrag                                                            1629
Ausbildung, Studium                                                            1006
Studentenjobs, Werkstudent                                                      990
Arbeitnehmerüberlassung                                                         829
Berufseinstieg/Trainee                                                          659
Freie Mitarbeit/Projektmitarbeit                                                154
Feste Anstellung, Berufseinstieg/Trainee                                        126
Praktikum, Studentenjobs, Werkstudent                                           108
Bachelor-/Master-/Diplom-Arbeiten                                               107
Handelsvertreter                                                            

In [207]:
#Contract Type in zwei Felder splitten:
df_contract_type = pd.read_excel("contract_type.xlsx")

In [208]:
df_contract_type.head(1)

,contract_type,Vertragsart,Befristung
0,Feste Anstellung,Anstellung,nicht bekannt


In [209]:
df = pd.merge(df, df_contract_type, on='contract_type', how='left')

In [210]:
df["Vertragsart"].isnull().sum()

0

In [211]:
df["Befristung"].isnull().sum()

0

# Arbeitgeberdaten

## Company Employee Count
* die (richtigen) Zahlen extrahieren aus Text

In [212]:
df["company_employee_count"].isnull().sum()

24863

In [213]:
# convert to string:
df["company_emp_count"] = df["company_employee_count"].astype("str")

In [214]:
df["company_emp_count"].dtype

dtype('O')

In [215]:
# df.company_employee_count.describe(percentiles = [.10, .25, .50, .75, .90]) 
# resources:
# https://www.geeksforgeeks.org/python-pandas-dataframe-describe-method/
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html

In [216]:
# flag all companies, that used gendering in the employee count field 
# "Mitarbeitende", "MitarbeiterInnen", "Mitarbeiter/innen", " Mitarbeiterinnen und Mitarbeiter", "Mitarbeiter und Mitarbeiterinnen"
df["company_emp_count_gendered"] = [1 if re.search(r'.*Mitarbeitende.*|.*MitarbeiterInnen.*|.*"Mitarbeiter/innen".*|.*Mitarbeiterinnen\sund\sMitarbeiter".*|.*Mitarbeiter\sund\sMitarbeiterinnen.*', field) else 0 for field in df["company_emp_count"].astype("str")]
df["company_emp_count_gendered"].value_counts()

0    50196
1      255
Name: company_emp_count_gendered, dtype: int64

In [217]:
replace1 = {"0-50": "50",
            "51-100": "100",
            "101-500": "500",
            "501-1000": "1000",
            "501-1.000": "1000",
            "298.683 (davon 174.663 in Deutschland)": "298683",
            "https://www.ey.com/de/": "10700",
            "8.000 in DE, 286.000 international": "286000",
            "http://www.kpmg.de/careers": "11700",
            "2000; 44.000 weltweit": "44000",
            "mehr als 11.000 Mitarbeiter an 21 Standorten in Deutschland": "11000",
            "http://www.adesso.de": "3700",
            "ca. 20.000 weltweit, ca. 8.000 in Deutschland": "20000",
            "http://www.hapeko.de": "105",
            "http://www.bauhaus.info/karriere": "34500",
            "über 70.000 Mitarbeiter deutschlandweit": "70000",
            "15.000 Mitarbeiter weltweit": "15000",
            "Bundesweit ca 7.500 Mitarbeiter an über 360 Standorten": "7500",
            "über 2.700 Mitarbeiter an 29 Standorte in der D-A-CH-DK Region": "2700",
            "über 7.900 weltweit (über 3.100 deutschlandweit)": "7900",
            "3358 (ohne Töchter)": "3358",
            "http://www.dataport.de/": "3000",
            "> 23.000 in Deutschland, > 48.000 in Europa": "48000",
            "mehr als 173.000 weltweit (23.675 in Deutschland)": "173000",
            "rund 70.000 Mitarbeiter deutschlandweit, rund 140.000 Mitarbeiter europaweit.": "140000",
            "über 900 Mitarbeiter an 13 Standorten (Abu Dhabi - Berlin - Budapest - Bukarest - Dubai - Düsseldorf - Frankfurt - Hamburg - München - Riad - Stuttgart - Wien - Zürich)": "900",
            "http://www.fiduciagad.de/": "4417",
            " Mitarbeiter in Deutschland": "",
            "http://www.swm.de": "9040",
            " Mitarbeiter (Deutschland)": "",
            "82.000 Mitarbeiter weltweit, 15.000 in Deutschland": "82000",
            "400-500": "500",
            "rund 12.200 Mitarbeiter weltweit, davon über 8.300 in Deutschland": "12200",
            "Weltweit: rund 100,000 Mitarbeiter (m_w_d)": "100000",
            "http://www.porsche.com": "32325",
            "http://www.de.capgemini.com": "211300",
            "https://bcgplatinion.com/de/": "170",
            "http://www.dis-ag.com/": "8500",
           }

In [218]:
df["company_emp_count"].replace(replace1, inplace=True)

In [219]:
# Internetadressen entfernen.
# df["company_emp_count"] = [re.sub(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', "", field) for field in df["company_employee_count"].astype("str")]

df["klammern"] = [re.search(r'\(.+\)', field).group(0) if re.search(r'\(.+\)', field) else "none" for field in df["company_emp_count"]]
df["klammern"].value_counts()

In [220]:
# Leerzeichen Kommas und Punkte, Jahreszahlen entfernen 
df["company_emp_count"] = [re.sub(r'(\.)', "", field) for field in df["company_emp_count"].astype("str")]
df["company_emp_count"] = [re.sub(r'(\s)', "", field) for field in df["company_emp_count"].astype("str")]
df["company_emp_count"] = [re.sub(r'(,)', "", field) for field in df["company_emp_count"].astype("str")]
df["company_emp_count"] = [re.sub(r'(2015)', "", field) for field in df["company_emp_count"].astype("str")]
df["company_emp_count"] = [re.sub(r'(2015)', "", field) for field in df["company_emp_count"].astype("str")]
df["company_emp_count"] = [re.sub(r'(2017)', "", field) for field in df["company_emp_count"].astype("str")]
df["company_emp_count"] = [re.sub(r'(2018)', "", field) for field in df["company_emp_count"].astype("str")]
df["company_emp_count"] = [re.sub(r'(2019)', "", field) for field in df["company_emp_count"].astype("str")]

In [221]:
def extractMax(input):
    #global maximal
    numbers = re.findall(r'\d+',input) 
    if len(numbers) >= 1: # [0].isdigit() #  isinstance(input, object):
        numbers = map(int,numbers) 
        maximal = max(numbers) 
    else:
        maximal = input
    return maximal

In [222]:
input =  "298683(davon174663inDeutschland)"
re.findall(r'\d+',input) 

['298683', '174663']

In [223]:
input= "ey"
extractMax(input)

'ey'

In [224]:
df["company_emp_count"] = df["company_emp_count"].apply(lambda x: extractMax(x))
df["company_emp_count"].value_counts()

nan                                                                     24863
1000                                                                     2888
50                                                                       1471
500                                                                      1272
298683                                                                    904
10700                                                                     751
100                                                                       563
286000                                                                    540
11000                                                                     469
200000                                                                    452
11700                                                                     437
3000                                                                      429
44000                                                           

In [225]:
replace2 = {"http://wwwdecapgeminicom": "211300",
            "http://wwwdecapgemini-consultingcom/karriere": "211300",
            "http://wwwhermesworldcom": "12000",
            "http://wwwsecunetcom": "500",
            "http://wwwtwt-gmbhde": "350",
            "http://wwwglatthaarcom": "500",
            "http://wwwkarcherde": "12300",
            "http://wwwedeka-verbundde/minden": "72189",
            "http://jobseos-karrierede/": "7500",
            "http://wwwfreudenberg-itde": "1200",
            "http://careersrochecom/germany": "94400",
            "http://wwwcontrolwarede": "760",
            "http://wwwrecaro-ascom": "2300",
}

In [226]:
df["company_emp_count"].replace(replace2, inplace=True)

In [227]:
# alles was keine Zahl ist in missing value umwandeln: 
df["company_emp_count"] = [field if field.isnumeric() else np.nan for field in df["company_emp_count"].astype("str")]

In [228]:
# convert to int:
df["company_emp_count"] = df["company_emp_count"].dropna().astype("int")

In [229]:
df["company_emp_count"].value_counts()

print(df["company_emp_count"].isnull().sum())
df["company_emp_count"].describe()


27329


count     23122.000000
mean      41530.009990
std       91960.743444
min           2.000000
25%         500.000000
50%        2800.000000
75%       15000.000000
max      510000.000000
Name: company_emp_count, dtype: float64

http://www.etas.com                                                         19
http://www.madsack.de/Karriere                                              19
http://www.expert.de                                                        19
http://www.wirkaufendeinauto.de                                             18
http://www.manufactum.de                                                    18
http://www.syngenio.com                                                     16
http://www.tempo-team.com                                                   16
http://www.fondsfinanz.de                                                   16
http://www.ltur.de                                                          16
http://www.engie-deutschland.de                                             16
http://www.bdk-bank.de                                                      16
http://www.creditplus.de                                                    16
http://www.dkfz.de                                                          16
http://wackerneusongroup.com                                                15
http://comdirect.de                                                         15
https://www.msc.com                                                         15
http://www.tq-group.com                                                     15
http://www.commerzbank.de/karriere                                          15
http://www.apollo.de                                                        15
http://www.medac.de                                                         15
http://karriere.juwi.de/                                                    14
http://www.westfalen-ag.de                                                  14
http://www.hamburgsud.com                                                   14
http://www.escrypt.com                                                      14
https://www.eagleburgmann.com/de                                            13
http://www.manpower.de                                                      13
http://www.hexal.de                                                         13
http://jobs.sportscheck.com/                                                13
http://www.fendt.com                                                        12
http://www.f-i-solutions-plus.de                                            12
http://www.bera.eu                                                          12
http://www.valeo.com                                                        12
http://www.alten-technology.com                                             12
https://www.tuev-nord-group.com/de/karriere/                                12
http://www.dental-akquise.de/                                               12
https://www.delphi-hr.de                                                    12
https://www.avantgarde-experts.de/de/ueber-uns/wir-als-arbeitgeber/         11
http://www.bghm.de                                                          11
https://www.leg-wohnen.de                                                   11
http://www.testotis.de                                                      11
http://www.quadriga-media.com                                               11
http://www.warema.de                                                        11
http://www.dirk.hoege.service.amv.de                                        11
http://www.schaeffler.com/carrers                                           11
http://www.rehau.com                                                        11
http://www.truma.com                                                        11
http://www.lucanet.com                                                      11
http://www.thieme.de                                                        10
http://www.fissler.de                                                       10
http://www.boehringer-ingelheim.de                                          10
http://www.griesemann-gruppe.de                                             10
http://www.versatel.de                                                      10
http://www.hansemerkur.de                                                   10

In [230]:
df[(df["company_emp_count"].isnull() == True)]["company_employee_count"].value_counts()

http://www.bankpower.de                                                    36
https://www.persona.de/                                                    32
http://meteor-ag.de                                                        26
http://www.etas.com                                                        19
http://www.madsack.de/Karriere                                             19
http://www.expert.de                                                       19
http://www.manufactum.de                                                   18
http://www.wirkaufendeinauto.de                                            18
http://www.ltur.de                                                         16
http://www.creditplus.de                                                   16
http://www.bdk-bank.de                                                     16
http://www.syngenio.com                                                    16
http://www.engie-deutschland.de                                 

## Company Sector

In [231]:
# df["company_sector"].value_counts()
df["intsinsector"] = [re.findall(r'\d+',sector) for sector in df["company_sector"].astype("str")]
print(df["intsinsector"].value_counts())

[]    50451
Name: intsinsector, dtype: int64


In [232]:
df.drop(["intsinsector"], inplace=True, axis=1)

## Company Location

In [233]:
df["company_location"].value_counts()

Düsseldorf                                                                                                         5390
Hamburg                                                                                                            2673
Stuttgart                                                                                                          2630
Berlin                                                                                                             2536
München                                                                                                            1984
Köln                                                                                                               1361
Frankfurt am Main                                                                                                   875
Frankfurt                                                                                                           838
Gerlingen Schillerhöhe                  

In [234]:
df[df['company_location'].str.contains('[\b\w]?(\d+[\.]\d\w)', na=False)]['company_location'].value_counts()

ca. 14.000 Mitarbeiter                                                          194
ca. 750 Interne MA und ca. 11.000 externe MA                                     13
über 10.000 Mitarbeiter/innen                                                    12
27.000                                                                           10
über 94.000 Mitarbeitende weltweit                                                6
über 70.000 Mitarbeiter deutschlandweit                                           5
Ca. 6.000 Mitarbeiter                                                             5
501-1.000                                                                         5
über 9.000 Mitarbeiter weltweit                                                   3
75.000 Mitarbeiterinnen und Mitarbeiter weltweit, 7.000 davon in Deutschland      2
2.200                                                                             2
weltweit mehr als 8.000 Mitarbeiter                                         

In [235]:
df[['company_emp_count']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50451 entries, 0 to 50450
Data columns (total 1 columns):
company_emp_count    23122 non-null float64
dtypes: float64(1)
memory usage: 788.3 KB


#### Werte von company_location nach company_emp_count übertragen:

In [236]:
df['company_emp_count'] = np.where(df['company_location'] == "2.200", 2200, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "weltweit mehr als 8.000 Mitarbeiter", 8000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "75.000 Mitarbeiterinnen und Mitarbeiter weltweit, 7.000 davon in Deutschland", 75000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "ca. 2.000", 2000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "über 10.000", 10000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "ca. 750 Interne MA und ca. 11.000 externe MA", 750, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "ca. 14.000 Mitarbeiter", 14000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "über 10.000 Mitarbeiter/innen", 10000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "27.000", 27000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "über 94.000 Mitarbeitende weltweit", 94000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "501-1.000", 1000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "Ca. 6.000 Mitarbeiter", 6000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "über 70.000 Mitarbeiter deutschlandweit", 70000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "über 9.000 Mitarbeiter weltweit", 9000, df['company_emp_count'] )

df['company_emp_count'] = np.where(df['company_location'] == "5+", 5, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "Rund 240 Mitarbeiter", 240, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "150 - 250", 250, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "rund 200 Mitarbeiterinnen und Mitarbeiter", 200, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "2000", 2000, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "150 Mitarbeiter", 150, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "450", 450, df['company_emp_count'] )
df['company_emp_count'] = np.where(df['company_location'] == "> 5000 Mitarbeiter", 5000, df['company_emp_count'] )

In [237]:
df[df['company_location'].str.contains('(?:(?<!\d)\d{3}(?!\d))', na=False)]['company_location'].value_counts()

ca. 14.000 Mitarbeiter                                                          194
ca. 750 Interne MA und ca. 11.000 externe MA                                     13
über 10.000 Mitarbeiter/innen                                                    12
27.000                                                                           10
LS101AB Leeds, Great Britain                                                     10
über 94.000 Mitarbeitende weltweit                                                6
Rund 240 Mitarbeiter                                                              6
450                                                                               5
501-1.000                                                                         5
über 70.000 Mitarbeiter deutschlandweit                                           5
150 - 250                                                                         5
Ca. 6.000 Mitarbeiter                                                       

# Missing Values

In [239]:
df.isna().sum()

unique_id                         0
title                             0
company                           0
company_link                      0
location                          0
contract_type                     0
wrk_time                          0
date                              0
job_url                           0
sections_str                      0
urls                              0
searches                          0
job_count                         0
search                            0
function                          0
region                            0
company_name                    571
company_location                809
company_sector                28196
company_rating                30016
company_rating_count          30150
company_job_count              1379
company_employee_count        24863
company_website               31608
company_metadatablock         15124
function_label_level2             0
function_label_level1             0
function_id_level_1         

In [240]:
# Jobs, die Text nur in sections1 stehen haben
df[df["section2"].isna()].sample(50)

,unique_id,title,company,company_link,location,contract_type,wrk_time,date,job_url,sections_str,...,KldB10-4,kldb10-2_label,kldb10-3_label,frauenanteil_mz14_kldb10_2,Frauenanteil_MZ14_kldb10_3,Bundesland,Vertragsart,Befristung,company_emp_count,company_emp_count_gendered
27380,27382,Kostencontroller (m/w/d),HanseMerkur Versicherungsgruppe,https://www.stepstone.de/cmp/de/HanseMerkur-Versicherungsgruppe-7510/jobs.html,Hamburg,Feste Anstellung,Vollzeit,2019-07-20,https://www.stepstone.de/stellenangebote--Kostencontroller-m-w-d-Hamburg-HanseMerkur-Versicherungsgruppe--4978453-inline.html?suid=1595f7df-489e-44b8-bbf0-4ccbc72c9bbf&rltr=4_4_100_dynrl_m_0_0_0,['Die HanseMerkur Versicherungsgruppe sucht einen Kostencontroller (m/w/d) Standort: Hamburg Die HanseMerkur Versicherungsgruppe ist ein finanziell solider mittelständischer Personenversicherer un...,...,7221.0,"Finanzdienstl.Rechnungsw.,Steuerberatung","Rechnungswesen, Controlling und Revision",0.603664,65.96,Hamburg,Anstellung,nicht bekannt,NaN,0
18227,18229,(Senior) Manager (m/w/d) Consulting - HCM,KPMG AG Wirtschaftsprüfungsgesellschaft,https://www.stepstone.de/cmp/de/KPMG-AG-Wirtschaftspruefungsgesellschaft-332/jobs.html,"Berlin, Bremen, Köln, Düsseldorf, Essen, Frankfurt am Main, Mannheim, München",Feste Anstellung,Vollzeit,2019-07-16,https://www.stepstone.de/stellenangebote--Senior-Manager-m-w-d-Consulting-HCM-Berlin-Bremen-Koeln-Duesseldorf-Essen-Frankfurt-am-Main-Mannheim-Muenchen-KPMG-AG-Wirtschaftspruefungsgesellschaft--40...,['Begleiten Sie KPMG bei den zukünftigen Herausforderungen unserer Kunden. Begeistern auch Sie sich für die Vielfalt unserer Fragestellungen - und machen Sie gemeinsam mit uns den Unterschied. Gem...,...,4311.0,Informatik- und andere IKT-Berufe,Informatik,0.145675,12.45,Bremen,Anstellung,nicht bekannt,11700.0,0
49779,49787,(Senior) Consultant (m/w/d) Forensic Investigation – Financial Advisory,Deloitte,https://www.stepstone.de/cmp/de/Deloitte-92380/jobs.html,"Frankfurt, Hamburg, München, Stuttgart","Feste Anstellung, Berufseinstieg/Trainee",Vollzeit,2019-07-18,https://www.stepstone.de/stellenangebote--Senior-Consultant-m-w-d-Forensic-Investigation-Financial-Advisory-Frankfurt-Hamburg-Muenchen-Stuttgart-Deloitte--4914020-inline.html?suid=2de4d58f-43e8-49...,"['(Senior) Consultant (m/w/d) Forensic Investigation – Financial Advisory Job Nummer: 19661 Standort: Frankfurt, Stuttgart Making an impact that matters. Unser Anspruch ist, jeden Tag das zu tun, ...",...,7139.0,"Berufe Unternehmensführung,-organisation",Unternehmensorganisation und -strategie,0.660914,54.20,Hamburg,Anstellung,nicht bekannt,286000.0,0
50002,50010,(Senior) Consultant (m/w/d) Forensic Investigation – Financial Advisory,Deloitte,https://www.stepstone.de/cmp/de/Deloitte-92380/jobs.html,"Frankfurt, Hamburg, München, Stuttgart","Feste Anstellung, Berufseinstieg/Trainee",Vollzeit,2019-07-18,https://www.stepstone.de/stellenangebote--Senior-Consultant-m-w-d-Forensic-Investigation-Financial-Advisory-Frankfurt-Hamburg-Muenchen-Stuttgart-Deloitte--4914020-inline.html?suid=89d34315-9eda-46...,"['(Senior) Consultant (m/w/d) Forensic Investigation – Financial Advisory Job Nummer: 19661 Standort: Frankfurt, Stuttgart Making an impact that matters. Unser Anspruch ist, jeden Tag das zu tun, ...",...,7139.0,"Berufe Unternehmensführung,-organisation",Unternehmensorganisation und -strategie,0.660914,54.20,Baden-Württemberg,Anstellung,nicht bekannt,286000.0,0
8195,8196,Key Account Manager technischer Vertrieb Sparte Profiltechnik / Maschinentechnik Region West (m/w/di,thyssenkrupp Infrastructure GmbH,https://www.stepstone.de/cmp/de/thyssenkrupp-Infrastructure-GmbH-165061/jobs.html,Essen,Feste Anstellung,Vollzeit,2019-07-17,https://www.stepstone.de/stellenangebote--Key-Account-Manager-technischer-Vertrieb-Sparte-Profiltechnik-Maschinentechnik-Region-West-m-w-di-Essen-thyssenkrupp-Infrastructure-GmbH--5655435-inline.h...,"['thyssenkrupp, das sind mehr als 161.000 Mitarbeiter in 79 Ländern und knapp € 

In [241]:
# Wie oft kommen einzelne Stellentitel vor? 
len(df["title"].unique())
# sehr heterogen, viele Titel kommen nur einmal oder selten vor, 29521 einzigartige Titel

29521

In [242]:
# Alle Stellen anzeigen, die in section6 einen Text haben:
# pd.options.display.max_colwidth = 50
#df[df["section6"].notna()] 
# sind alles Stellen bei Stepstone selbst! sind scheinbar die einzigen, die alle Felder nutzen 
# ggfs. die Stepstone - Anzeigen harmonisisieren auf das Format der anderen

In [243]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50451 entries, 0 to 50450
Data columns (total 52 columns):
unique_id                     50451 non-null int64
title                         50451 non-null object
company                       50451 non-null object
company_link                  50451 non-null object
location                      50451 non-null object
contract_type                 50451 non-null object
wrk_time                      50451 non-null object
date                          50451 non-null object
job_url                       50451 non-null object
sections_str                  50451 non-null object
urls                          50451 non-null object
searches                      50451 non-null object
job_count                     50451 non-null int64
search                        50451 non-null object
function                      50451 non-null int64
region                        50451 non-null int64
company_name                  49880 non-null object
company_loc

# Spalten umbenennen

In [244]:
df.rename(columns= {'function': 'function_level_2',
                    'function_id_level_1': 'function_level_1',
                    'function_label_level1': 'function_level1_label',
                    'function_label_level2': 'function_level2_label',
                    'location': 'job_location',
                    'company': 'job_company',
                    'KldB10-2': 'kldb10_2',
                    'KldB10-3': 'kldb10_3',
                    'KldB10-4': 'kldb10_4',
                    'kldb10-2_label': 'kldb10_2_label',
                    'kldb10-3_label': 'kldb10_3_label',
                   }, inplace=True)

In [245]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50451 entries, 0 to 50450
Data columns (total 52 columns):
unique_id                     50451 non-null int64
title                         50451 non-null object
job_company                   50451 non-null object
company_link                  50451 non-null object
job_location                  50451 non-null object
contract_type                 50451 non-null object
wrk_time                      50451 non-null object
date                          50451 non-null object
job_url                       50451 non-null object
sections_str                  50451 non-null object
urls                          50451 non-null object
searches                      50451 non-null object
job_count                     50451 non-null int64
search                        50451 non-null object
function_level_2              50451 non-null int64
region                        50451 non-null int64
company_name                  49880 non-null object
company_loc

# Abspeichern des Datensatzes

In [246]:
df.to_csv("aufbereitet_2019-12-15.csv", index=False, sep=";")

In [ ]:
# df = pd.read_csv("aufbereitet_2019-08-18.csv", sep=";")